# Showcase for a video

showcase one of the Mongo database instance failures and how it's continued to work.

In [1]:
from pymongo import MongoClient

def print_member_status(client):
    db = client.admin
    rs_status = db.command({'replSetGetStatus': 1})

    for m in rs_status['members']:
        print(m['name'], m['stateStr'])

In [2]:
client = MongoClient('mongodb://localhost:27017,localhost:27018,localhost:27019/?replicaSet=foo')
print_member_status(client)

localhost:27017 PRIMARY
localhost:27018 SECONDARY
localhost:27019 SECONDARY


In [3]:
# Insert some documents to test collection

db = client.test
db.test.insert_one({'name': 'test1'})
db.test.insert_one({'name': 'test2'})
db.test.insert_one({'name': 'test3'})

# Print the documents in the collection
for doc in db.test.find():
    print(doc)

{'_id': ObjectId('681dfbee3227ff9807e5b165'), 'name': 'test1'}
{'_id': ObjectId('681dfbee3227ff9807e5b166'), 'name': 'test2'}
{'_id': ObjectId('681dfbee3227ff9807e5b167'), 'name': 'test3'}


In [4]:
# Kill the primary server manually then run this
print_member_status(client)

# Print the documents in the collection
for doc in db.test.find():
    print(doc)

# Another node becomes primary instantly and you can still read the documents

localhost:27017 SECONDARY
localhost:27018 PRIMARY
localhost:27019 SECONDARY
{'_id': ObjectId('681dfbee3227ff9807e5b165'), 'name': 'test1'}
{'_id': ObjectId('681dfbee3227ff9807e5b166'), 'name': 'test2'}
{'_id': ObjectId('681dfbee3227ff9807e5b167'), 'name': 'test3'}


In [5]:
print_member_status(client)

# The killed not becomes unreachable

localhost:27017 (not reachable/healthy)
localhost:27018 PRIMARY
localhost:27019 SECONDARY


In [6]:
# Start the killed node and it will become secondary
print_member_status(client)

localhost:27017 SECONDARY
localhost:27018 PRIMARY
localhost:27019 SECONDARY


In [ ]:
# remove the documents
db.test.delete_many({})